# Example: estimating selection coefficients

In [1]:
from isweep import *

### This is the main function you use to simulate IBD segments from a hard selective sweep.

In [2]:
help(simulate_ibd_isweep)

Help on function simulate_ibd_isweep in module isweep.coalescent:

simulate_ibd_isweep(n, s, p0, Ne, long_ibd=2.0, short_ibd=1.0, random_walk=True, one_step_model='a', tau0=0, sv=-0.01, ploidy=2, record_dist=True, pairwise_output=True)
    ibd segments from a coalescent with selection

    Parameters
    ----------
    n : int
        Sample size (individuals)
    s : float
        Selection coefficient
    p0 : float
        Variant frequency at generation 0
    Ne : dict
        Effective population sizes
    long_ibd, short_ibd : float
        cM length threshold
    random_walk : bool
        True for random walk
    one_step_model : str
        'm', 'a', 'd', or 'r'
    tau0 : int
        Generation when neutrality begins
    sv: float
        Allele frequency of standing variation
        (Default -0.01 will assume de novo sweep)
    ploidy : int
        1 for haploid or 2 for diploid
    record_dist : bool
        To save tract length and coalescent time distributions or not to 

### This is the main function you optimize over to estimate the selection coefficient.

In [3]:
help(chi2_isweep)

Help on function chi2_isweep in module isweep.inference:

chi2_isweep(s, p0, Ne, n, obs, ab, one_step_model='a', tau0=0, sv=-0.01, ploidy=2)
    Chi-squared statistic for sweep model (unlabeled)

    Parameters
    ----------
    s : float
        Selection coefficient
    p0 : float
        Variant frequency at generation 0
    Ne : dict
        Effective population sizes
    n : int
        Pairs sample size
    obs : array-like
        Observed counts for ibd segment bins (unlabeled)
    ab : array-like
        Increasing floats in centiMorgans
    one_step_model : str
        'm', 'a', 'd', or 'r'
    tau0 : int
        Generation at which neutrality begins
    sv: float
        Allele frequency of standing variation
        (Default -0.01 will assume de novo sweep)
    ploidy : int
        1 for haploid or 2 for diploid

    Returns
    -------
    float
        Goodness-of-fit statistic



### Making effective population sizes file

In [4]:
help(make_constant_Ne)
# make_exponential_Ne('decreasing-N3000-R0.04.ne', 3000, [300], [0.04])
# extend_Ne

Help on function make_constant_Ne in module isweep.utilities:

make_constant_Ne(file, size, maxg)
    Create *.ne file for constant size population

    Parameters
    ----------
    file: str
        Output file name
    size : float
        Effective population size
    maxg : int
        Maximum generation

    Returns
    -------
    NoneType
        Writes a *.ne file



In [159]:
make_constant_Ne('constant-10k.ne', 10000, 2000)

### Setup

In [165]:
# selection coefficient
s = 0.03

p=float(0.5) # allele freq
Ne=read_Ne('constant-10k.ne') # demo history
model='m'
long_ibd=3.0
ab=[long_ibd,np.inf]
nsamples=200

# diploids
ploidy=2
msamples=ploidy*nsamples
N=msamples*(msamples-1)/2-msamples

# # haploidy
# ploidy=1
# msamples=ploidy*nsamples
# N=msamples*(msamples-1)/2

### Point estimate a selection coefficient

In [166]:
out=simulate_ibd_isweep(
    nsamples,
    s,
    p,
    Ne,
    long_ibd,
    long_ibd,
    one_step_model=model,
    ploidy=ploidy,
)
ibd=out[0][0]
se = minimize_scalar(
    chi2_isweep,
    args=(p,Ne,N,(ibd,),ab,model,0,-0.01,ploidy),
    bounds=(0,0.5),
    method='bounded'
).x
print('true selection coefficient')
print(s)
print('estimate selection coefficient')
print(se)

true selection coefficient
0.03
estimate selection coefficient
0.03644378240753399


### Selection coefficient confidence interval

In [167]:
K = 100 # bootstrap number

ests1 = []
for k in range(K):
    if ((k+1) % 10) == 0:
        print(k)
    out=simulate_ibd_isweep(
    nsamples,
    s,
    p,
    Ne,
    long_ibd,
    long_ibd,
    one_step_model=model,
    ploidy=ploidy,
    )
    ibd=out[0][0]
    se = minimize_scalar(
        chi2_isweep,
        args=(p,Ne,N,(ibd,),ab,model,0,-0.01,ploidy),
        bounds=(0,1.),
        method='bounded'
    ).x
    ests1.append(se)

# +- 2 * sigma is close to the symmetric 95% confidence interval
# assuming data is normally distributed
(np.mean(ests1)-2*np.std(ests1),np.mean(ests1)+2*np.std(ests1))

9
19
29
39
49
59
69
79
89
99


(0.00749035618472255, 0.04496696320884709)